In [1]:
import mysql.connector as sql
from sqlalchemy import create_engine
import pandas as pd
import json

In [2]:
with open('config.json', 'r') as openfile:
# Reading from json file
    json_object = json.load(openfile)

host=json_object['host']
user=json_object['user']
passw=json_object['passw']
port=json_object['port']
db=json_object['db']
# prod credentials
prdhost=json_object['prdhost']
prduser=json_object['prduser']
prdpassw=json_object['prdpassw']
prdport=json_object['prdport']
prddb=json_object['prddb']
# differentiate factor for red/blue
is_multi_queue=json_object['is_multi_queue']

In [3]:
if is_multi_queue ==1:
    factor_blue=str(json_object['blue_factor'])
    factor_red=str(json_object['red_factor'])
else:
    factor_blue="0"
    factor_red="0"


In [4]:
try:
    prod = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}"
    .format(host=prdhost, db=prddb, user=prduser, pw=prdpassw,port=prdport))
    
    with prod.connect() as prod_con:
        program_prod=pd.read_sql_query("SELECT * FROM afiniti_3.program;",prod)
        activemodel_prod=pd.read_sql_query("SELECT * FROM afiniti_3.activemodelunits;",prod)
        team_rotation = pd.read_sql_query("SELECT Date,( case when Queue1 = -31 then 'Red' when Queue1 = -32 then 'Blue' else Team end)  as Team, Queue1 FROM siriusxmussatmap.`de.teamrotation` where date = curdate();",prod)
except:
    print("Error while connecting to prod db...")
        
prod_con.close()
prod.dispose()

activemodel_prod.head()

,program_id,modelunit_id,tenant_id,active_datetime,evaluator_instance
0,468,1395,0,2021-06-08 04:18:45,default
1,468,1396,0,2021-06-08 04:18:45,default
2,468,1397,0,2021-06-08 04:18:45,default
3,468,1398,0,2021-06-08 04:18:45,default
4,468,1399,0,2021-06-08 04:18:45,default


In [5]:
# Create SQLAlchemy engine to connect to MySQL Database
archival = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}"
.format(host= host, db=db, user=user, pw=passw,port=port))

In [6]:
with archival.connect() as con:

   # if is_multi_queue ==1:
    evaluator_test1  = pd.read_sql_query("""
        SELECT * 
        FROM etl_dev.evaluator_test1
        where id>(

            select IFNULL(max(id),0) as id
            from
            (
                select max(id) as id from etl_dev.eval_view_blue_test
                union all
                select max(id) as id from etl_dev.eval_view_red_test
            ) temp
        )
        """, archival)
   # else:
   #     evaluator_test1 = pd.read_sql_query("SELECT * FROM etl_dev.evaluator_test1 where date(insert_time) = curdate()", archival)
    
    activemodelunit_red = pd.read_sql_query("SELECT program_id,modelunit_id+"+factor_red+" as modelunit_id,tenant_id,active_datetime,evaluator_instance FROM activemodelunits_red_test;", archival)
    program_red = pd.read_sql_query('SELECT * FROM program_red_test;', archival)
    activemodelunit_blue = pd.read_sql_query("SELECT program_id,modelunit_id+"+factor_blue+" as modelunit_id,tenant_id,active_datetime,evaluator_instance From activemodelunits_blue_test;", archival)
    program_blue = pd.read_sql_query('SELECT * FROM program_blue_test;', archival)
    

In [7]:
program_red.head()

,program_id,program_name,program_predicate,algorithm_name,sensor_key,program_rank,generated_on
0,6,SXM_CA_Retention,"VDN_IN(1,2,3)",diagonal,1,1,NaT
1,7,SXMUS_Retention_SIM,"VDN_IN(116,112,115,113,114)&SKILL_ANY(19,20,21...",diagonal,SXMUS_Retention_SIM,2,NaT
2,9,SXMUS_Retention_SIM,"VDN_IN(116,112,115,113,114)&SKILL_ANY(19,20,21...",diagonal,SXMUS_Retention_SIM,1,NaT
3,10,SXM_CA_GENERAL,"VDN_IN(10,14,46,94,98,102)",diagonal,SXM_CA_GENERAL,2,NaT
4,11,SXM_CA_SALES,"VDN_IN(106,90,86,78,74,70,54,30)",diagonal,SXM_CA_SALES,3,NaT


In [8]:
sql_df = evaluator_test1
sql_df.columns=['id', 'evaluator_id', 'insert_time', 'calgroup_bandwidth']

In [9]:
sql_df.head()

,id,evaluator_id,insert_time,calgroup_bandwidth
0,2,default_2021_05_26-13_59_16,2021-05-26 19:55:06,"{\n ""calgroup_bandwidth"": [\n {\n ""Ba..."
1,3,default_2021_05_27-13_59_16,2021-05-27 19:55:06,"{\n ""calgroup_bandwidth"": [\n {\n ""Ba..."
2,4,default_2021_05_28-13_59_16,2021-05-28 16:55:06,"{\n ""calgroup_bandwidth"": [\n {\n ""Ba..."


In [10]:
#'evaluator_id', 'insert_time'
column_names = ["id", "evaluator_id", "insert_time","modelunit_id", "calgroup_bandwidth"]

df_final = pd.DataFrame(columns = column_names)

In [11]:
sql_df.head()

,id,evaluator_id,insert_time,calgroup_bandwidth
0,2,default_2021_05_26-13_59_16,2021-05-26 19:55:06,"{\n ""calgroup_bandwidth"": [\n {\n ""Ba..."
1,3,default_2021_05_27-13_59_16,2021-05-27 19:55:06,"{\n ""calgroup_bandwidth"": [\n {\n ""Ba..."
2,4,default_2021_05_28-13_59_16,2021-05-28 16:55:06,"{\n ""calgroup_bandwidth"": [\n {\n ""Ba..."


In [12]:
sql_df['calgroup_bandwidth']=sql_df['calgroup_bandwidth'].replace('\n','', regex=True).replace(' ', '',regex = True)
sql_df.head()

,id,evaluator_id,insert_time,calgroup_bandwidth
0,2,default_2021_05_26-13_59_16,2021-05-26 19:55:06,"{""calgroup_bandwidth"":[{""Bandwidth"":[],""CPID"":..."
1,3,default_2021_05_27-13_59_16,2021-05-27 19:55:06,"{""calgroup_bandwidth"":[{""Bandwidth"":[],""CPID"":..."
2,4,default_2021_05_28-13_59_16,2021-05-28 16:55:06,"{""calgroup_bandwidth"":[{""Bandwidth"":[],""CPID"":..."


In [13]:
data_dict=dict(sql_df['calgroup_bandwidth'])
data_dict.keys()

dict_keys([0, 1, 2])

In [14]:
try:
    for i in range(0,len(sql_df)):
        #print(sql_df.iloc[i])
        df = dict(sql_df.iloc[i])
        _id = df['id']
        evaluator_id = df['evaluator_id']
        insert_time = df['insert_time']
        #print(_id,evaluator_id,insert_time)
        st = data_dict[i]
        d = json.loads(st)
        ky=dict(d)
        ky=list(ky.keys())[0]
        print(ky)
        data = d[ky]
        for dt in data:
            #print(dt)
            df_final = df_final.append({
            "id" : _id,
            "evaluator_id" : evaluator_id,
            "insert_time"  : insert_time,
            "calgroup_bandwidth":str(dt),
            "modelunit_id": dt['MUID']}, ignore_index=True)
except:
     print("Error while parsing json")

        
#df_final.to_sql('eval_view_test', archival, index=False,if_exists='replace', index_label="RowNum", chunksize = 1000)

calgroup_bandwidth
calgroup_bandwidth
calgroup_bandwidth


In [15]:
df_final.head()

,id,evaluator_id,insert_time,modelunit_id,calgroup_bandwidth
0,2,default_2021_05_26-13_59_16,2021-05-26 19:55:06,191,"{'Bandwidth': [], 'CPID': 2, 'MUID': 191, 'Pro..."
1,2,default_2021_05_26-13_59_16,2021-05-26 19:55:06,190,"{'Bandwidth': [], 'CPID': 1, 'MUID': 190, 'Pro..."
2,2,default_2021_05_26-13_59_16,2021-05-26 19:55:06,196,"{'Bandwidth': [], 'CPID': 2, 'MUID': 196, 'Pro..."
3,2,default_2021_05_26-13_59_16,2021-05-26 19:55:06,832001,"{'Bandwidth': [{'CGID': -1, 'MAX': 9.1234, 'MI..."
4,2,default_2021_05_26-13_59_16,2021-05-26 19:55:06,5000620,"{'Bandwidth': [{'CGID': -1, 'MAX': 9.1234, 'MI..."


In [16]:
def generate_rb_multiqueue(activemodelunit,program,df_final,team,activemodel_prod):
    try:
        join_with_model_unit = pd.merge(df_final,activemodel_prod, on='modelunit_id' , how = 'inner')
        join_with_program = pd.merge(join_with_model_unit,activemodelunit, on='modelunit_id' , how = 'inner')
        #join_with_program=join_with_program[['id','evaluator_id','insert_time','calgroup_bandwidth','program_id_x']]
        join_with_program= pd.merge(join_with_program,program,left_on='program_id_x',right_on='program_id',how='inner')
        join_with_program=join_with_program[['id','evaluator_id','insert_time','calgroup_bandwidth']]

        if len(join_with_program)>0:
            join_with_program['calgroup_bandwidth'] = join_with_program.groupby(['id','insert_time','evaluator_id'])['calgroup_bandwidth'].transform(lambda x : ','.join(x))
            join_with_program = join_with_program.drop_duplicates() 
            
            join_with_program['calgroup_bandwidth'] = " {'calgroup_bandwidth' : [" + join_with_program['calgroup_bandwidth'] + "]}"
            join_with_program['calgroup_bandwidth']=join_with_program['calgroup_bandwidth'].replace("'",'"',regex=True) 
    
            
    
            join_with_program.to_sql('eval_view_'+team+'_test', archival, index=False,if_exists='append', index_label="RowNum", chunksize = 1000)
        else:
            print("No Data for "+team)
    except:
          print("An exception occurred for "+team)
            


In [17]:
if is_multi_queue ==1:
    generate_rb_multiqueue(activemodelunit_red,program_red,df_final,'red',activemodel_prod)
    generate_rb_multiqueue(activemodelunit_blue,program_blue,df_final,'blue',activemodel_prod)
else:
    team = team_rotation['team'][0]
    if team_rotation == 'None':
        pass
    else:
        if team == 'Red':
            generate_rb_multiqueue(activemodelunit_red,program_red,df_final,'red',activemodel_prod)
        elif team == 'Blue':
            generate_rb_multiqueue(activemodelunit_blue,program_blue,df_final,'blue',activemodel_prod)
    
con.close()
archival.dispose()

No Data for red
No Data for blue


In [18]:
team_rotation['Team'][0]

'Red'

In [19]:
"""if len(join_with_program)>0:
    join_with_program['calgroup_bandwidth'] = join_with_program.groupby(['id','insert_time','evaluator_id'])['calgroup_bandwidth'].transform(lambda x : ','.join(x))
    join_with_program = join_with_program.drop_duplicates() 
      
    join_with_program['calgroup_bandwidth'] = " {'calgroup_bandwidth' : [" + join_with_program['calgroup_bandwidth'] + "]}"
    join_with_program['calgroup_bandwidth']=join_with_program['calgroup_bandwidth'].replace("'",'"',regex=True) 
     
    """

'if len(join_with_program)>0:\n    join_with_program[\'calgroup_bandwidth\'] = join_with_program.groupby([\'id\',\'insert_time\',\'evaluator_id\'])[\'calgroup_bandwidth\'].transform(lambda x : \',\'.join(x))\n    join_with_program = join_with_program.drop_duplicates() \n      \n    join_with_program[\'calgroup_bandwidth\'] = " {\'calgroup_bandwidth\' : [" + join_with_program[\'calgroup_bandwidth\'] + "]}"\n    join_with_program[\'calgroup_bandwidth\']=join_with_program[\'calgroup_bandwidth\'].replace("\'",\'"\',regex=True) \n     \n    '